In [1]:
import os, random, json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import SequentialSampler, SubsetRandomSampler
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.loader import DataLoader as GeoDataLoader

import optuna
from metrics import metrics_from_preds
from preprocess import *

# Carga de datos

In [2]:
"""
# Para secuenciales
train_loader_seq = DataLoader(VideoDataset(vids_training, labels_training), batch_size=32, shuffle=False)
val_loader_seq   = DataLoader(VideoDataset(vids_val, labels_val), batch_size=32, shuffle=False)
"""

'\n# Para secuenciales\ntrain_loader_seq = DataLoader(VideoDataset(vids_training, labels_training), batch_size=32, shuffle=False)\nval_loader_seq   = DataLoader(VideoDataset(vids_val, labels_val), batch_size=32, shuffle=False)\n'

In [3]:
def set_seed(seed=42):
    SEED = seed
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Controla el hash aleatorio de Python
    os.environ["PYTHONHASHSEED"] = str(seed)

In [4]:
set_seed(42)

In [12]:
device = torch.device("cpu")
print("Usando dispositivo:", device)

Usando dispositivo: cpu


In [5]:
vids_training, labels_training, vids_val, labels_val, vids_test, labels_test = load_data()

In [6]:
def build_graph(frames, label):
    """
    Convierte un video (16 x 258) en un grafo:
    - 16 nodos (uno por frame).
    - Cada nodo con 258 features.
    - Aristas entre frames consecutivos.
    """
    x = torch.tensor(frames, dtype=torch.float32)  # (16, 258)

    # Conexiones secuenciales (cadena temporal)
    edge_index = []
    for i in range(len(frames) - 1):
        edge_index.append([i, i+1])
        edge_index.append([i+1, i])
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()  # (2, E)

    y = torch.tensor([label], dtype=torch.long)  # etiqueta del grafo
    return Data(x=x, edge_index=edge_index, y=y)

In [7]:
class VideoGraphDataset(InMemoryDataset):
    def __init__(self, videos, labels, transform=None):
        self.videos = videos
        self.labels = labels
        super().__init__('.', transform)

        self.data_list = [build_graph(v, l) for v, l in zip(videos, labels)]

    def len(self):
        return len(self.data_list)

    def get(self, idx):
        return self.data_list[idx]

In [8]:
set_seed()

train_graph_dataset = VideoGraphDataset(vids_training, labels_training)
val_graph_dataset   = VideoGraphDataset(vids_val, labels_val)


# Permutación fija para tener "shuffle" pero reproducible
g = torch.Generator().manual_seed(42)
perm = torch.randperm(len(train_graph_dataset), generator=g).tolist()
train_sampler = SubsetRandomSampler(perm)

train_loader_graph = GeoDataLoader(train_graph_dataset, batch_size=32, shuffle=False, num_workers=0, sampler=train_sampler)
val_loader_graph   = GeoDataLoader(val_graph_dataset, batch_size=32, shuffle=False, num_workers=0)

/tmp/ipykernel_37810/1024000706.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  x = torch.tensor(frames, dtype=torch.float32)  # (16, 258)


# Modelo GCN (Baseline)

In [9]:
class GCNModel(nn.Module):
    """GCN simple para clasificación de 4 clases.
    Entrada: grafo de un frame/video -> salida: logits (B, 4)
    """
    def __init__(self, in_channels=258, hidden_channels=256, num_classes=4, layers=2, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        # Primer bloque
        self.layers = nn.ModuleList([GCNConv(in_channels, hidden_channels)])
        # Capas intermedias (layers - 1)
        for _ in range(layers - 1):
            self.layers.append(GCNConv(hidden_channels, hidden_channels))
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index, batch):
        # x: nodos (N, in_channels), edge_index: aristas, batch: asignación de nodos a grafos
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        x = self.dropout(x)
        x = global_mean_pool(x, batch)  # pooling global
        return self.lin(x)

In [10]:
# ========= GCN =========
def optimize_gcn(train_loader_graph, val_loader_graph, n_trials=50, max_epochs=80, save_prefix="hpo_gcn"):
    set_seed()
    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
    all_trials = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def objective(trial):
        hidden  = trial.suggest_categorical("hidden",[96,128,160,192])
        layers  = trial.suggest_int("layers", 2, 5)
        drop    = trial.suggest_float("dropout", 0.0, 0.5)
        lr      = trial.suggest_float("lr", 5e-5, 5e-3, log=True)
        wd      = trial.suggest_float("weight_decay", 1e-10, 1e-3, log=True)

        model = GCNModel(hidden_channels=hidden, layers=layers, dropout=drop).to(device)
        crit = nn.CrossEntropyLoss()
        opt  = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)

        best_val = 0.0
        for epoch in range(max_epochs):
            model.train()
            for batch in train_loader_graph:
                batch = batch.to(device)
                opt.zero_grad()
                out = model(batch.x, batch.edge_index, batch.batch)
                loss = crit(out, batch.y)
                loss.backward(); opt.step()
            # val
            model.eval()
            y_true, y_pred = [], []
            with torch.no_grad():
                for batch in val_loader_graph:
                    batch = batch.to(device)
                    pred = model(batch.x, batch.edge_index, batch.batch).argmax(1)
                    y_true.extend(batch.y.cpu().numpy()); y_pred.extend(pred.cpu().numpy())
            val_acc = metrics_from_preds(y_true, y_pred)["accuracy"]
            if val_acc > best_val: best_val = val_acc
            trial.report(val_acc, epoch)
            if trial.should_prune(): raise optuna.TrialPruned()

        all_trials.append({"trial": trial.number, "params": trial.params, "best_val_acc": best_val})
        return best_val

    study.optimize(objective, n_trials=n_trials)
    with open(f"{save_prefix}_all.json","w") as f: json.dump(all_trials,f,indent=2)
    with open(f"{save_prefix}_best.json","w") as f: json.dump({"best_params":study.best_params,
                                                               "best_value":study.best_value},
                                                              f,
                                                              indent=2)
    return study

In [11]:
study_gcn = optimize_gcn(train_loader_graph, val_loader_graph, n_trials=3, max_epochs=100) #n_trials = 40

[I 2025-10-26 21:41:05,547] A new study created in memory with name: no-name-a3224ffa-8cfc-4206-a2d8-c471a8a5d8a3
[I 2025-10-26 21:41:32,395] Trial 0 finished with value: 0.8555555555555555 and parameters: {'hidden': 160, 'layers': 3, 'dropout': 0.282559779642901, 'lr': 0.0004939871586762675, 'weight_decay': 1.6168342504508736e-06}. Best is trial 0 with value: 0.8555555555555555.
[I 2025-10-26 21:41:54,482] Trial 1 finished with value: 0.8555555555555555 and parameters: {'hidden': 96, 'layers': 3, 'dropout': 0.050677846187594355, 'lr': 8.677690387452583e-05, 'weight_decay': 2.7364999437208665e-08}. Best is trial 0 with value: 0.8555555555555555.
[I 2025-10-26 21:42:26,596] Trial 2 finished with value: 0.8944444444444445 and parameters: {'hidden': 192, 'layers': 4, 'dropout': 0.4197928569413311, 'lr': 0.0017240556232222808, 'weight_decay': 7.909224199438224e-05}. Best is trial 2 with value: 0.8944444444444445.


In [ ]:
print("Mejor resultado:", study_gcn.best_value)
print("Mejores hiperparámetros:", study_gcn.best_params)